# Clase 7: Ajustando distribuciones a los datos - casos especiales

Prof. Tito Homem-de-Mello

In [ ]:
#install.packages('fitdistrplus')
#install.packages('gglot2')
install.packages('goft')


In [ ]:
library(fitdistrplus)
library(ggplot2)
library(goft)


## Miremos un ejemplo con datos de tiempos de fallas de equipamientos

data <- read.csv("fallas.csv")
head(data)

In [ ]:
X <- data$Tiempos.entre.fallas

In [ ]:
bin <- 2
plotfig <- ggplot(data,aes(X))+
        theme(plot.title = element_text(size = 20),
              axis.title.y=element_text(size = 20),
              axis.title.x=element_text(size = 20),
              axis.text.y=element_text(size = 15),
              axis.text.x=element_text(size = 15))        )
 
p <- plotfig+geom_histogram(aes(y = after_stat(count /(sum(count)))), binwidth=bin,color="black", fill="gray")+
       labs(x="tiempo",
            y="frecuencia",
            title=paste0("Histograma de los datos"))           
p 

In [ ]:
descdist(X)

### Ajustemos algunas distribuciones teóricas

In [ ]:

a_exp <- fitdist(X, "exp")
summary(a_exp)

a_gamma <- fitdist(X, "gamma")
summary(a_gamma)

a_lnorm <- fitdist(X, "lnorm")
summary(a_lnorm)

a_weib <- fitdist(X, "weibull")
summary(a_weib)

plot.legend <- c( "Weibull", "Gamma", "Lognormal","Exponential")
denscomp(list(a_gamma,a_lnorm,a_weib,a_exp), fitcol=c(2,4,6,1), fitlty=1, legendtext=plot.legend)

## Determinemos la bondad de ajuste para las 4 distribuciones

### Empecemos con el PP-Plot

In [ ]:
ppcomp(list(a_lnorm,a_gamma,a_weib,a_exp))

### Grafiquemos el QQ-Plot

In [ ]:
qqcomp(list(a_lnorm,a_gamma,a_weib,a_exp))


## Ahora veamos los tests estadísticos

In [ ]:
gofval <- gofstat(list(a_lnorm,a_gamma,a_weib,a_exp),fitnames=c("lnorm","gamma","weib","exp"))
gofval

### Valores-$p$ para test chi-cuadrado

In [ ]:
pvalue <- data.frame(Estadistica="Valor-p",  
                     Lognormal=gofval$chisqpvalue["lnorm"],Gamma=gofval$chisqpvalue["gamma"],
                     Weibull=gofval$chisqpvalue["weib"],
                     Exp=gofval$chisqpvalue["exp"])     
pvalue

In [ ]:
ntest <- normal_test(X) #Use shapiro.test(X) si el nro. de datos no está entre 10 y 400
ltest <- lnorm_test(X)
gtest <- gamma_test(X)
wtest <- weibull_test(X)
etest <- exp_test(X)
pvalue<-data.frame(Estadistica="Valor-p",Normal=ntest$p.value,
                   Lognormal=ltest$p.value,Gamma=gtest$p.value,Weibull=wtest$p.value,
                   Exp=etest$p.value)     
pvalue

## ¿Qué hacer?....